<a href="https://colab.research.google.com/github/aicuai/GenAI-Steam/blob/main/Stable_Audio_Open.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stable Audio Open: Google Colab Notebook, by AICU Inc.

Stable Audio Open: Research Paper (22 Jul, 2024)

https://stability.ai/news/stable-audio-open-research-paper


[Stability AI、オープンウェイトなテキスト楽曲生成「Stable Audio Open」公開。Google Colabで無料で体験！](https://note.com/aicu/n/nbfabd78acb8e)


[X@AICUai](https://x.com/AICUai/status/1815421664106250261)

In [ ]:
!pip install einops
!pip install stable_audio_tools

In [ ]:
import torch
import torchaudio
from einops import rearrange
from stable_audio_tools import get_pretrained_model
from stable_audio_tools.inference.generation import generate_diffusion_cond

device = "cuda" if torch.cuda.is_available() else "cpu"

# Download model
model, model_config = get_pretrained_model("stabilityai/stable-audio-open-1.0")
sample_rate = model_config["sample_rate"]
sample_size = model_config["sample_size"]

model = model.to(device)

# Set up text and timing conditioning
conditioning = [{
    "prompt": "128 BPM tech house drum loop",
    "seconds_start": 0,
    "seconds_total": 30
}]

# Generate stereo audio
output = generate_diffusion_cond(
    model,
    steps=100,
    cfg_scale=7,
    conditioning=conditioning,
    sample_size=sample_size,
    sigma_min=0.3,
    sigma_max=500,
    sampler_type="dpmpp-3m-sde",
    device=device
)

# Rearrange audio batch to a single sequence
output = rearrange(output, "b d n -> d (b n)")

# Peak normalize, clip, convert to int16, and save to file
output = output.to(torch.float32).div(torch.max(torch.abs(output))).clamp(-1, 1).mul(32767).to(torch.int16).cpu()
torchaudio.save("output.wav", output, sample_rate)
